# Rule Unit 작성 예시

본 문서는 1차년도 6월에 개발된 RuleUnit 클래스를 활용하여 룰 유닛을 작성하고 실행하는 과정을 설명합니다.

⚠︎ 주의사항: 코드를 변경하며 작업할 때는, 사본을 생성한 후 작업해주십시오.

## 룰 유닛 작성하기
이 부분에서는 기존의 IFC 파일에 종속적이었던 실행 흐름에서 벗어나, 새로이 정의한 RuleUnit 구조에 기반하여 룰에 포함된 항목을 작성하는 방법을 다룹니다.<br><br>
본 예시에서는 KDS 24 14 21 4.6.5.1(5) 를 룰 유닛으로 작성합니다.

먼저, Rule 작성에 필요한 프로그램을 불러오기 위해 Github에서 아래의 주소가 가리키는 파일 모음을 불러옵니다. (Github Repository를 clone합니다.)

In [ ]:
!git clone https://github.com/KU-HIAI/tomok.git

Cloning into 'tomok'...
remote: Enumerating objects: 175, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 175 (delta 54), reused 85 (delta 31), pack-reused 58
Receiving objects: 100% (175/175), 451.83 KiB | 4.00 MiB/s, done.
Resolving deltas: 100% (62/62), done.


필요한 파일을 불러온 다음에는 현재 작업 경로를 tomok의 파일이 있는 위치로 이동합니다.<br><br>
아래의 코드 블록을 실행하면, 작성자의 기본 작업 경로가 필요한 파일이 있는 위치로 이동됩니다.

In [ ]:
cd tomok

/content/tomok


## 룰 유닛 작성해보기
지금까지의 내용을 바탕으로 룰 유닛을 작성해봅시다.

새로운 룰 유닛을 작성하기 위해서는, 아래 코드 블럭에서 클래스 이름, 메타정보에 해당하는 변수, 실행 함수의 이름과 내용을 수정하면 됩니다.

In [ ]:
import math
from typing import List

import tomok

# 작성하는 룰에 맞게 클래스 이름 수정 (KDS249011_04020406_02)
class KDS249011_04020406_02 (tomok.RuleUnit):

    # 아래 클래스 멤버 변수에 할당되는 값들을 작성하는 룰에 맞게 수정
    priority = 1   # 건설기준 우선순위
    author = 'Seohyun Jin'  # 작성자명
    ref_code = 'KDS 24 90 11 4.2.4.6 (2)' # 건설기준문서
    ref_date = '2021-04-12'  # 디지털 건설문서 작성일
    doc_date = '2023-09-27'  # 건설기준문서->디지털 건설기준 변환 기준일
    title = '탄성패드 설계'    # 건설기준명

    #
    description = """
    교량 기타시설설계기준 (한계상태설계법)
    4. 설계
    4.2 받침
    4.2.4 포트받침
    4.2.4.6 탄성패드 설계
    (2)
    """
    # https://dillinger.io/ 표와 이미지 랜더링 확인 사이트
    # 이미지 링크 변환 사이트 https://www.somanet.xyz/2017/06/blog-post_21.html
    # 건설기준문서내용(text)
    content = """
    ####  4.2.4.6 탄성패드 설계
    (2) 탄성패드는 극한한계상태에서 다음 조건을 만족해야 한다.
    $$N_{Sd}\leq N_{Rd}$$&#160;&#160;&#160;&#160;&#160;(4.2-25)
    여기서, $$E_{b}$$ 는 탄성중합체의 체적탄성계수로 일반적으로 2,000MPa를 사용한다
    $$N_{Rd}$$ 는 탄성패드의 설계강도로 다음과 같다.
    $$N_{Rd} = \frac{1}{1.3}\times \frac{\pi }{4}\times d^{2}\times f_{e,k}$$&#160;&#160;&#160;&#160;&#160;(4.2-26)
    여기서, $$f_{e,k}$$ = 탄성중합체의 허용압축응력(60MPa)

    """
    # 플로우차트(mermaid)
    flowchart = """
    flowchart TD
    subgraph Python_Class
    A[탄성패드의 설계];
    B["KDS 24 90 11 4.2.4.6 (2)"];
    A ~~~ B
    end

		subgraph Variable_def;
		VarIn1[/입력변수: 탄성패드의 설계강도/];
		VarIn2[/입력변수: 탄성중합체의 허용압축응력/];
		VarIn3[/입력변수: 직경/];
		VarIn4[/입력변수: 수평력/];

    VarIn1 & VarIn2 & VarIn3 & VarIn4

		end

		Python_Class ~~~ Variable_def;
		Variable_def--->K--->L--->M

		K["<img src='https://latex.codecogs.com/svg.image?N_{Rd}=\frac{1}{1.3}\times\frac{\pi}{4}\times&space;d^{2}\times&space;f_{e,k}'>--------------------------------------------------------"];
		L["<img src='https://latex.codecogs.com/svg.image?N_{Sd}\leq&space;N_{Rd}'>--------------------------------------------------------"];
		M(["Pass or Fail"])
    """

    # 작성하는 룰에 맞게 함수 이름과 내용을 수정,
    @tomok.rule_method
    def Design_Strength_Of_Elastic_Pads(fONRd, fId, fINsd, fIfek) -> bool:
        """탄성패드 설계

        Args:
            fONRd (float): 탄성패드의 설계강도
            fId (float): 직경
            fINsd (float): 수평력
            fIfek (float): 탄성중합체의 허용압축응력

        Returns:
            bool: 교량 기타시설설계기준 (한계상태설계법)  4.2.4.6 탄성패드 설계 (2)의 통과 여부
        """
        import math
        fONRd = ((math.pi)*(fId**2)*fIfek)/(1.3*4)

        if fINsd <= fONRd :
          return "Pass"
        else :
          return "Fail"


작성한 룰 유닛은 아래의 코드 블럭과 같이 생성하여, 작성자가 임의로 검증을 수행할 수 있습니다.

In [ ]:
my_RuleUnit = KDS249011_04020406_02()

In [ ]:
fONRd = None
fId = 2000
fINsd = 200
fIfek = 60

In [ ]:
Rule_Review_Result = my_RuleUnit.Design_Strength_Of_Elastic_Pads(fONRd, fId, fINsd, fIfek)
# 해당건설기준 항목 를 통과하는가?

print("RuleUnit Review Result: {}".format(Rule_Review_Result))

RuleUnit Review Result: Pass


<br><br>
아래의 코드를 통해, 룰 유닛의 content(건설기준 항목의 실제 내용)의 markdown 렌더링 결과를 확인할 수 있습니다.

In [ ]:
my_RuleUnit.render_markdown()


    ####  4.2.4.6 탄성패드 설계
    (2) 탄성패드는 극한한계상태에서 다음 조건을 만족해야 한다.
    $$N_{Sd}\leq N_{Rd}$$&#160;&#160;&#160;&#160;&#160;(4.2-25)
    여기서, $$E_{b}$$ 는 탄성중합체의 체적탄성계수로 일반적으로 2,000MPa를 사용한다
    $$N_{Rd}$$ 는 탄성패드의 설계강도로 다음과 같다.
    $$N_{Rd} = rac{1}{1.3}	imes rac{\pi }{4}	imes d^{2}	imes f_{e,k}$$&#160;&#160;&#160;&#160;&#160;(4.2-26)
    여기서, $$f_{e,k}$$ = 탄성중합체의 허용압축응력(60MPa)

    